In [ ]:
import csv
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
import nltk
from itertools import chain
nltk.download('punkt')
import re
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Функция, которая всё чистит и токенизирует:

In [ ]:
def clean_up(pp):
    sent = sent_tokenize(pp.lower())
    tok = list(map(lambda x: word_tokenize(x), sent))
    tok = list(chain(*tok))
    depunct = list(filter(lambda x: x not in punctuation, tok))
    depunct = list(filter(lambda x: x not in '«»–', depunct))
    return depunct

In [ ]:
with open('/content/gdrive/MyDrive/progs/raw_hw2.txt', 'r', encoding='utf-8') as f:
    body = f.read()
    words = clean_up(body)

In [ ]:
words[2]

'семинаре'

In [ ]:
with open('pretagging_hw2.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=' ')
    for rr in words:
        writer.writerow([rr])

Потом я разметила этот текст в csv. Я использовала тагсет Universal Dependencies, потому что уже знакома с ним после ПИЛДов и для этого тагсета есть понятное объяснение всяких случаев вроде путаницы между DET и PRON. А ещё он подходит для разных языков, а я как-то верю в возможную унификацию признаков между языками

https://universaldependencies.org/treebanks/ru_syntagrus/index.html

## Тэггеры

### pymorphy

- https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html

In [ ]:
#!pip install pymorphy2

In [ ]:
from pymorphy2 import MorphAnalyzer

In [ ]:
morpher = MorphAnalyzer()

In [ ]:
pm_pos = list(map(lambda x: morpher.parse(x)[0].tag.POS if morpher.parse(x)[0].tag.POS!= N, words))

### *natasha*

https://github.com/natasha/natasha

In [ ]:
#!pip install natasha

In [ ]:
from natasha import (MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Segmenter,
    Doc)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(' '.join(words))
doc.segment(segmenter) 

In [ ]:
doc.tag_morph(morph_tagger)

In [ ]:
natasha_pos = [doc.tokens[i].pos for i in range(len(words))]

In [ ]:
natasha_pos[1]

'ADJ'

### deeppavlov

http://docs.deeppavlov.ai/en/master/features/models/morphotagger.html

In [ ]:
#!pip install deeppavlov

In [ ]:
#!python -m deeppavlov install morpho_ru_syntagrus_pymorphy

In [ ]:
from deeppavlov import build_model, configs

model = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy, download=True)

In [ ]:
parced = model(words)
parced[:3]

['1\tна\tADP\t_\n',
 '1\tпоследнем\tADJ\tCase=Loc|Degree=Pos|Gender=Masc|Number=Sing\n',
 '1\tсеминаре\tNOUN\tAnimacy=Inan|Case=Loc|Gender=Masc|Number=Sing\n']

In [ ]:
pavlov_pos = list(map(lambda x: re.split('\t',x)[2], parced))

И наташа, и павлов работают с тагсетом Universal Dependencies, а вот у pymorphy другие тэги:

In [ ]:
set(pavlov_pos)

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'SCONJ',
 'VERB',
 'X'}

In [ ]:
set(natasha_pos)

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 'X'}

In [ ]:
set(pm_pos)

{'ADJF',
 'ADJS',
 'ADVB',
 'COMP',
 'CONJ',
 'INFN',
 'NOUN',
 'NPRO',
 'NUMR',
 None,
 'PRCL',
 'PRED',
 'PREP',
 'PRTF',
 'VERB'}

In [ ]:
translator = {'ADJF':['ADJ','DET'], 'ADJS':'ADJ', 'ADVB':'ADV','COMP':['ADJ','ADV'],
              'CONJ':['SCONJ','CCONJ'],'INFN':'VERB','NPRO':'PRON','NUMR':'NUM',
              'PRCL':'PART','PRED':'ADV','PREP':'ADP','PRTF':['VERB','ADJ']}

In [ ]:
pm_pos_ud = list(map(lambda x: translator[x] if x in translator.keys() else x, pm_pos))

In [ ]:
with open('/content/gdrive/MyDrive/progs/tagging_hw2.csv', 'r', encoding='utf-8') as g:
  reader = csv.reader(g)
  tags = [i for i in reader if len(i)>0]
  pos = [k[1] for k in tags if len(k)>1]

In [ ]:
len(pm_pos_ud) == len(pm_pos) == len(natasha_pos) == len(pavlov_pos)==len(pos)

True

In [ ]:
len(tags)

328

In [ ]:
def accuracy(test):
    trues = sum([True for k in zip(pos,test) if k[0] in k[1]])
    return trues/len(pos)

In [ ]:
accuracy(pavlov_pos)

0.926829268292683

In [ ]:
accuracy(natasha_pos)

0.7012195121951219

In [ ]:
accuracy(pm_pos_ud)

0.899390243902439

Лучший тэггер - Павлов.


**n-граммы:**
- не + [token+] VERB
(чтобы классификатор был чувствителен к отрицанию - понравилось/не_понравилось)


- не + [token+] + ADJ
(чтобы классификатор был чувствителен к отрицанию - лучший/не\_самый_лучший)

- ADV+ADJ
(мне кажется, мнго оценочных выражений устроены по такой модели - выше среднего, хуже ожидаемого и тд)

In [ ]:
pavlov_pairs = list(map(lambda x: re.split('\t',x)[1:3], parced))

In [ ]:
pavlov_pairs[19]

['этой', 'DET']

In [140]:
def chunker(text):
    #text - список пар (word,pos_tag)
    n_grams=list()
    for k in range(len(text)-2):
        if text[k][0]=='не' and text[k+2][1] in ['ADJ','VERB']:
            n_grams.append(text[k:k+3])
    for k in range(len(text)-1):
        if text[k][0]=='не' and text[k+1][1] in ['ADJ','VERB']:
          n_grams.append(text[k:k+2])
        elif text[k][1]=='ADV' and text[k+1][1]=='ADJ':
          n_grams.append(text[k:k+2])
    return n_grams

Проверяем:

In [143]:
piece = 'Фильм не очень хороший, тотально отвратительный. Я не заценил'

In [144]:
par = model(clean_up(piece))
pavlov_prs = list(map(lambda x: re.split('\t',x)[1:3], par))
chunker(pavlov_prs)

[[['не', 'PART'], ['очень', 'ADV'], ['хороший', 'ADJ']],
 [['очень', 'ADV'], ['хороший', 'ADJ']],
 [['тотально', 'ADV'], ['отвратительный', 'ADJ']],
 [['не', 'PART'], ['заценил', 'VERB']]]

А в прошлую домашку я это не вставлю, потому что она была на английских текстах, а никто не предупредил, что нужны русские